## California Supervised Learning - MinMax Scaler

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import collections

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import silhouette_samples
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import ward, dendrogram, complete, average, single

import geopandas as gpd
import pygeos
import gpdvega 

import altair as alt


In [ ]:
#Load in US shape data
counties = gpd.read_file('/work/ca-county-boundaries/CA_Counties/CA_Counties_TIGER2016.shp')
counties['county_name'] = counties['NAME'].str.lower()
counties.head()

In [ ]:
#Load in California data
ca_df = pd.read_csv('/work/cleaned-csvs/ca_counties_full_dataset.csv')

ca_df.columns

In [ ]:
# Check for sparse columns
nullseries=ca_df.isna().sum()
nullseries[nullseries > 0]

In [ ]:
#Data prep for unsupervised learning tasks

#Let's just look at one year of data, 2018
ca_df = ca_df[ca_df['year']==2018]

#Before removing text columns, create a mapper for county names
ca_df = ca_df.reset_index()
county_list_mapper = list(ca_df['county_name'])
county_dict_mapper = (ca_df[['county_name']].to_dict('index'))

#Identify and drop non-numeric fields
non_numeric = ca_df.select_dtypes(exclude='number').columns.to_list()
ca_df = ca_df.drop(columns=non_numeric)


#Remove fields that are numeric, but not meaningful features of the county
ca_df = ca_df.drop(columns = {'state','county', 'longitude',
       'latitude','year','index'})


ca_df.shape

In [ ]:
#Store the column names for later use
ca_df_columns = ca_df.columns
ca_df_columns

### Feature Scaling
We'll use min-max normalization since all of our features are on different scales, and also numeric.

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(ca_df)

# verify that there are no missing values and inspect data
print('missing values:', pd.DataFrame(scaled_data).isna().sum().sum())
pd.DataFrame(scaled_data)


In [ ]:
scaled_data.shape

### K-Means Clustering

We will run K-Means with multiple n_clusters values so we can use the elbow method. To determine the optimal number of clusters, we have to select the value of k at the “elbow”, or the point after which the inertia start decreasing in a linear fashion. 

Inertia is the sum of squared distances of samples to their closest cluster center (SSE)

We will try two methods of initializing K-Means, the default, which is 'k-means++', which selectes inital cluster centers in a "smart way", and 'random' which choses initial centroids at random.

We will also look at two metrics to check the quality of our clusters:

#### Davies_Bouldin Index
This index signifies the average ‘similarity’ between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves. Zero is the lowest possible score. Values closer to zero indicate a better partition.

#### Calinski-Harabasz Index
The index is the ratio of the sum of between-clusters dispersion and of within-cluster dispersion for all clusters (where dispersion is defined as the sum of distances squared). The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.



In [ ]:
k_range = [*range(2, 19)]

#Calculate inertia scores using the default initialization 'k-means++'
inertia_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42,init ='k-means++').fit(scaled_data)
    inertia_scores.append(kmeans.inertia_)
    kmeans.fit(scaled_data)
    ch_score = metrics.calinski_harabasz_score(scaled_data,kmeans.labels_)
    db_score = metrics.davies_bouldin_score(scaled_data,kmeans.labels_)
    print('k: {}, ch_score: {}, db_score {} '.format(k,ch_score,db_score))


Our Calinksi-Harabasz scores are all low.

inertia_ provides the sum of the squared error, SSE. We will plot this to find "elbow"


In [ ]:
plt.plot(k_range, inertia_scores, '-o')
plt.ylabel('SSE')
plt.xlabel('Number of clusters (k)')
plt.title('The Elbow Method using "k-means++" initialization ')

Seeing an elbow at about 3, let's try using the random initialization

In [ ]:
inertia_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42,init='random').fit(scaled_data)
    inertia_scores.append(kmeans.inertia_)
    kmeans.fit(scaled_data)
    ch_score = metrics.calinski_harabasz_score(scaled_data,kmeans.labels_)
    db_score = metrics.davies_bouldin_score(scaled_data,kmeans.labels_)
    print('k: {}, ch_score: {}, db_score {} '.format(k,ch_score,db_score))



Our Calinski-Harabasz scores are still low. Let's see if the elbow shows us anything different...

In [ ]:
# plt.plot(k_range, inertia_scores, '-o')
# plt.ylabel('SSE')
# plt.xlabel('Number of clusters (k)')
# plt.title('The Elbow Method using "random" initialization')

Both seem to show a change in descent at about 5 clusters. Let's try using silhouette analysis.

From https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py:

Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.



In [ ]:
# Code from https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a
# Per the article, a good number of clusters will have a well above 0.5 silhouette 
# average score as well as all of the clusters have higher than the average score

for i, k in enumerate(range(2,11)):
    #fig, (ax1, ax2) = plt.subplots(1, 2)
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    labels = km.fit_predict(scaled_data)
    centroids = km.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(scaled_data, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    print('Average silhouette score for {} clusters is {}'.format(k,avg_score))
    
    #plt.tight_layout()
    plt.suptitle(f'Silhouette analysis using k = {k}',
                 fontsize=16, fontweight='semibold', y=1.05);


None of our clusters have a score higher than 0.5. Based on this, I see two possible conclusions:
1. K-Means is maybe not the right algorithm for us
3. Maybe there are too many dimensions? Can try running PCA on the data, then K-Means, will do this later

### Agglomerative Clustering
Agglomerative clustering performs a bottom up approach where each observation starts in its own cluster and clusters are successively merged together.

There are 4 types of linkages we will consider:
- Single: minimum distance between clusters
- Complete: maximum distance between clusters
- Average: average distance between clusters
- Ward: difference between:
- - The total within-cluster sum of squares for the two clusters seperately
- - and
- - The within-cluster sum of squares resulting from merging the two clusters

We are using Ward's Method to start because it tends to create equal sized clusters and is effective for noisy data. 

The output of the ward function is as follows:
- Column 1 and 2 are child nodes
- Column 3 is distance
- Column 4 is the number of leaf nodes merged

Every node 58 and higher are nodes that are created from merges. Node id 58 is created from the first line of the ward result, node id 59 is the second line, etc.

In [ ]:
plt.figure(figsize=(10,6))
cls1 = ward(scaled_data)
dendrogram(cls1)#,orientation='left')
plt.title('Dendrogram using Ward Linkage')
plt.show()

print(cls1)

Based on the dendrogram we are going to assign the data to 3 clusters, so now we will run the clusters to get the assignment value

In [ ]:
cls_w = AgglomerativeClustering(n_clusters=3, linkage='ward')
cls_assignment_w = cls_w.fit_predict(scaled_data)
cls_assignment_w

In [ ]:
#Zip together the mapper and the cluster assignment
county_cluster = list(zip(county_list_mapper, list(cls_assignment_w)))

cluster_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_ward_cluster'])
cluster_df


In [ ]:
#Show the clusters on a map
ca_all = pd.merge(counties,cluster_df,how='inner',on='county_name')

In [ ]:
agglom_ward = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_ward_cluster:N'
).properties(title='Agglomerative Clusters - Ward Linkage - 2018 Data')
agglom_ward

Out of curiousity, let's see if we get similar clusterings if we use different agglomerative clustering linkage methods 

In [ ]:
#Complete linkage method
plt.figure(figsize=(10,6))
cls2 = complete(scaled_data)
dendrogram(cls2)#,orientation='left')
plt.title('Dendrogram using Complete Linkage')
plt.show()

print(cls2)

With complete linkage it seems that 5 clusters are more appropriate

In [ ]:

cls_c = AgglomerativeClustering(n_clusters=5, linkage='complete')
cls_assignment_c = cls_c.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_c)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_complete_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Average Linkage Method
plt.figure(figsize=(10,6))
cls3 = average(scaled_data)
dendrogram(cls3)#,orientation='left')
plt.title('Dendrogram using Average Linkage')
plt.show()

print(cls3)

With average linkage it seems that 5 clusters are appropriate

In [ ]:

cls_a = AgglomerativeClustering(n_clusters=5, linkage='average')
cls_assignment_a = cls_a.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_a)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_average_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Single linkage method
plt.figure(figsize=(10,6))
cls4 = single(scaled_data)
dendrogram(cls4)#,orientation='left')
plt.title('Dendrogram using Single Linkage')
plt.show()

print(cls4)

Using single linkage 2 clusters seems most appropriate

In [ ]:
cls_s = AgglomerativeClustering(n_clusters=2, linkage='single')
cls_assignment_s = cls_s.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_s)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'agglom_single_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

Show the rest of the agglomerative clustering maps

In [ ]:
#Show Complete Linkage Cluster Map

complete = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_complete_cluster:N'
).properties(title='Agglomerative Clusters - Complete Linkage - 2018 Data')

complete

In [ ]:
#Show average linkage cluster map
average = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_average_cluster:N'
).properties(title='Agglomerative Clusters - Average Linkage - 2018 Data')
average

In [ ]:
#Show single linkage cluster map
single = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='agglom_single_cluster:N'
).properties(title='Agglomerative Clusters - Single Linkage - 2018 Data')
single


### Back to K-Means...
Since agglomerative clustering methods generated results with 3 and 5 clusters (among others), out of curiousity let's try this with K-Means and see if our clustered counties are similar. Note that this is not the best tool for us to use given what we already found out about our K-Means clusters

In [ ]:
kmeans_3 = KMeans(n_clusters=3, random_state=42).fit(scaled_data)
kmeans_5 = KMeans(n_clusters=5, random_state=42).fit(scaled_data)

In [ ]:
#Add kmeans with 3 clusters to our dataframe
county_cluster = list(zip(county_list_mapper, list(kmeans_3.labels_)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'kmeans_3_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Add kmeans with 5 clusters to our dataframe
county_cluster = list(zip(county_list_mapper, list(kmeans_5.labels_)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'kmeans_5_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
ca_all.head()

In [ ]:
#Show K-Means with 3 clusters map
kmeans3_chart = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='kmeans_3_cluster:N'
).properties(title='K-Means With 3 Clusters - 2018 Data')
kmeans3_chart  

And for comparison, the agglomerative clustering with ward linkage
(I can't get them to print side by side, I am getting an error on cell output size)

In [ ]:
agglom_ward

These are pretty close! They only differ by one county: Tehama

In [ ]:
#Show K-Means with 5 clusters map
kmeans5_chart = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='kmeans_5_cluster:N'
).properties(title='K-Means With 5 Clusters - 2018 Data')
kmeans5_chart

This one is pretty similar to complete linkage. Just has a larger "Bay Area" cluster. 
Need to check out Alpine County - it is often on its own.

### DBSCAN
DBSCAN works well with datasets that have more complex cluster shapes. Clusters represent areas in the data space that are more dense with data points, while being separated by regions that are empty, or at least much less densely populated. This is done using connected components from graph theory. 

The eps parameter is the maximum distance between two samples for one to be considered as in the neighborhood of the other.

The min_samples parameter is the number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

DBSCAN labels data as noise by setting it's cluster value to -1, so we will need to keep an eye out for this.

Also, DBSCAN clustering is not completely deterministic so the results may change over different runs.

In [ ]:
#Since our data set is so small, we will go with 2 for min_samples, this means 
#we need two counties to create a cluster.

#Let's check which values of eps work well for our dataset
#If values are all -1 then eps is too small and everything is an outlier
#If values are all set to 1 cluster than eps is too large

for eps in np.linspace(1.0,2.0,11):
    dbscan = DBSCAN(eps = eps, min_samples = 2)
    cls_dbscan = dbscan.fit_predict(scaled_data)

    counter=collections.Counter(cls_dbscan)

    print('eps is: ',eps)
    print(counter)


When eps is 1.4 we get 4 clusters and 10 outliers. 


When eps is 1.5 we get 3 clusters and 7 outliers.


Now, what if we allow clusters to contain just one county? Then there are no outliers, but potentially many clusters. Let's try the same analysis to check for the best eps value

In [ ]:
for eps in np.linspace(1.0,2.0,11):
    dbscan = DBSCAN(eps = eps, min_samples = 1)
    cls_dbscan = dbscan.fit_predict(scaled_data)

    counter=collections.Counter(cls_dbscan)

    print('eps is: ',eps)
    print(counter)


If we compare this to the result for min_samples = 1, we see it is the same result for eps of 1.4 and 1.5 except that instead of being called an outlier, counties were created into their own clusters. Let's graph this so we can more easily see each singular county on its own.

In [ ]:
dbscan_1_4 = DBSCAN(eps = 1.4, min_samples = 1)
cls_dbscan_1_4 = dbscan_1_4.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_dbscan_1_4)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'dbscan_1_4_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
dbscan_1_5 = DBSCAN(eps = 1.5, min_samples = 1)
cls_dbscan_1_5 = dbscan_1_5.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_dbscan_1_5)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'dbscan_1_5_cluster'])

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Show DBSCAN with eps = 1.4 cluster map
dbscan_1_4_chart = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip=['NAME','dbscan_1_4_cluster'],
    color='dbscan_1_4_cluster:N'
).properties(title='DBSCAN with eps = 1.4 - 2018 Data')
dbscan_1_4_chart

In [ ]:
#Show DBSCAN with eps = 1.5 cluster map
dbscan_1_5_chart = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip=['NAME','dbscan_1_5_cluster'],
    color='dbscan_1_5_cluster:N'
).properties(title='DBSCAN with eps = 1.5 - 2018 Data')
dbscan_1_5_chart

DBSCAN does not seem to be a good choice for us. It is generally making one large cluster and lots of small clusters.

### PCA
We are working with a lot of features in California. Maybe this is impacting the ability to create useful clusters. Let's try PCA to help identify the most relevant features and to reduce the dimensionality so we can try clustering again.

In [ ]:
pca = PCA(2)
pca_data = pca.fit_transform(scaled_data)

Let's plot and check the variance of the components using a variance ratio plot

In [ ]:
plt.figure(figsize=(10,10))
var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
lbls = [str(x) for x in range(1,len(var)+1)]
plt.bar(x=range(1,len(var)+1), height = var, tick_label = lbls)
plt.xlabel('Principal Component')
plt.ylabel('Variance %')
plt.title('PCA Explained Variance')
plt.show()



In [ ]:
#Plot the magnitude of each feature value for the 2 principal components
# fig, ax = plt.subplots(figsize=(20, 14))
# plt.imshow(pca.components_[0:2], interpolation = 'none', cmap = 'plasma')
# feature_names=list(ca_df_columns)
# plt.xticks(np.arange(-0., len(feature_names), 1) , feature_names, rotation = 90, fontsize=10)
# plt.yticks(np.arange(0., 2, 1), ['First PC', 'Second PC'], fontsize = 16)
# plt.colorbar()

This is really hard to see horizontally, let's switch the orientation...

In [ ]:
#Plot the magnitude of each feature value for the 2 principal components
fig, ax = plt.subplots(figsize=(20, 18))
plt.imshow(pca.components_[0:2].T, interpolation = 'none', cmap = 'plasma')
feature_names=list(ca_df_columns)
plt.yticks(np.arange(-0., len(feature_names), 1) , feature_names, fontsize=10)
plt.xticks(np.arange(0., 2, 1), ['First PC', 'Second PC'], rotation = 90, fontsize = 16)
plt.colorbar()

It's kind of hard to use the colors in this plot for actually gauging importance. Let's use the components_ method to find the features that provided the most variance for each of the principle components. Here are the top 5 in principle component 1:

In [ ]:
pca_df = pd.DataFrame(pca.components_.T, index=ca_df_columns)
pca_df['PC-1'] = abs(pca_df[0])
pca_df['PC-2'] = abs(pca_df[1])

pca_df['PC-1'].nlargest(n=10)

And the top 10 most important features for principle component 2:

In [ ]:
pca_df['PC-2'].nlargest(n=10)

Here are the 20 features that showed the least variance for PC1. Note that this includes all the FEMA variables except for fire:

In [ ]:
pca_df['PC-1'].nsmallest(n=20)

Here are the 20 features that showed the least variance for PC2. This list includes all of the FEMA variables:

In [ ]:
pca_df['PC-2'].nsmallest(n=20)

#### Biplot
The cosine of the angle between any two variable markers (vectors) is the coefficient of correlation between those variables.
Note - this only holds if first two PC capture 80% of variance which is NOT the case here.

The cosine of the angle between a vector and the axis for a given PC is the coefficient of correlation between those two variables.

In [ ]:
#pca.components_[0:2].T, interpolation = 'none', cmap = 'plasma')
#feature_names=list(ca_df_columns)

# [x for x in (list(ca_df_columns))]

# list(range(0,len(ca_df_columns)))
# list(zip(list(range(0,len(ca_df_columns))),[x for x in (list(ca_df_columns))]))

# coastal_flag                  0.419181
# median_home_value             0.234739
# median_income                 0.226722
# democrat_pct                  0.224378
# median_rent                   0.221641
# american_independent_pct      0.213186
# republican_pct                0.211032
# educational_attainment        0.209305
# perc_asian                    0.194298
# per_capita_personal_income 

# perc_65_over                          0.305379
# perc_white                            0.280679
# perc_hispanic                         0.267592
# perc_poverty                          0.236784
# per_capita_num_vehicle_theft          0.224469
# registered_pct                        0.216534
# total_precip_amt                      0.210019
# per_capita_nonfarm_proprieter_jobs    0.201308
# max_temp                              0.192144
# avg_temp   

# [22,36,34,1,35,3,2,37,43,15,47,40,46,39,30,0,12,21,11,9]



In [ ]:
#From SIADS 543
feature_subset_count = 20

### Feel free to use this routine to plot your own biplots!
def biplot(score, coeff, maxdim, pcax, pcay, labels=None):
    zoom = 0.5
    pca1=pcax-1
    pca2=pcay-1
    xs = score[:,pca1]
    ys = score[:,pca2]
    n = min(coeff.shape[0], maxdim)
    width = 2.0 * zoom
    scalex = width/(xs.max()- xs.min())
    scaley = width/(ys.max()- ys.min())
    text_scale_factor = 1.3
        
    fig = plt.gcf()
    fig.set_size_inches(9, 9)
    
    plt.scatter(xs*scalex, ys*scaley, s=9)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,pca1], coeff[i,pca2],
                  color='b',alpha=0.9, head_width = 0.03 * zoom) 
        if labels is None:
            plt.text(coeff[i,pca1]* text_scale_factor, 
                     coeff[i,pca2] * text_scale_factor, 
                     "Var"+str(i+1), color='g', ha='center', va='center')
        else:
            plt.text(coeff[i,pca1]* text_scale_factor, 
                     coeff[i,pca2] * text_scale_factor, 
                     labels[i], color='black', ha='center', va='center')
    
    plt.xlim(-zoom,zoom)
    plt.ylim(-zoom,zoom)
    plt.xlabel("PC{}".format(pcax))
    plt.ylabel("PC{}".format(pcay))
    plt.grid()

plt.figure()

#Instead of taking first 10 variables, pick specific variables
#feature_subset = slice(0, feature_subset_count, 1)
#feature_subset= [22,3,34,1,35,47,40,46,39,30]
feature_subset= [22,36,34,1,35,3,2,37,43,15,47,40,46,39,30,0,12,21,11,9]

#Get my labels instead of the first 10 labels
biplot(pca_data, np.transpose(pca.components_[0:2, feature_subset]),
       #feature_subset_count, 1, 2, labels=feature_names[feature_subset])
       feature_subset_count, 1, 2, labels=[feature_names[x] for x in feature_subset])

print("explained_variance_ratio:", pca.explained_variance_ratio_)
print("sum of explained variance ratios:", np.sum(pca.explained_variance_ratio_))
print("singular values:", pca.singular_values_)  


### K-Means after PCA

Now we will train our model based on the new features generated by PCA

In [ ]:
k_range = [*range(2, 19)]

#Calculate inertia scores using the default initialization 'k-means++'
inertia_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42,init ='k-means++').fit(pca_data)
    inertia_scores.append(kmeans.inertia_)
    kmeans.fit(scaled_data)
    ch_score = metrics.calinski_harabasz_score(scaled_data,kmeans.labels_)
    db_score = metrics.davies_bouldin_score(scaled_data,kmeans.labels_)
    print('k: {}, ch_score: {}, db_score {} '.format(k,ch_score,db_score))


Let's try the elbow method again

In [ ]:
plt.plot(k_range, inertia_scores, '-o')
plt.ylabel('SSE')
plt.xlabel('Number of clusters (k)')
plt.title('The Elbow Method after PCA ')

The elbow is much more prounounced at 3 after PCA

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42).fit(pca_data)

In [ ]:
#Here's our data labeled by cluster
centers = np.array(kmeans.cluster_centers_)
#print(centers)

colors={0:'steelblue',1:'orange',2:'red'}

pca_label = kmeans.fit_predict(pca_data)
plt.figure(figsize=(10,10))
uniq = np.unique(pca_label)
for i in uniq:
   plt.scatter(pca_data[pca_label == i , 0] , pca_data[pca_label == i , 1] , label = i,c=colors[i])
plt.scatter(centers[:,0], centers[:,1], marker="x", color='k')
#This is done to find the centroid for each clusters.
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('California Counties After PCA, By K-Means Cluster')
plt.show()

In [ ]:

county_cluster = list(zip(county_list_mapper, list(pca_label)))

new_df = pd.DataFrame(county_cluster, columns=['county_name', 'kmeans_pca_cluster'])
new_df

ca_all = ca_all.merge(new_df, how = 'inner', on = 'county_name')

In [ ]:
#Show K-Means after PCA clusters map
kmeans_pca_chart = alt.Chart(ca_all).mark_geoshape().encode(
    tooltip='NAME',
    color='kmeans_pca_cluster:N'
).properties(title='K-Means With 3 Clusters on PCA Results - 2018 Data')
kmeans_pca_chart

Compare this to our prior kmeans map with 3 clusters...they are the same

In [ ]:
kmeans3_chart

### Write out data for Cluster Analysis

In [ ]:
output_fields = ['STATEFP','COUNTYFP','NAMELSAD','county_name','agglom_ward_cluster','agglom_complete_cluster',
                'agglom_average_cluster','agglom_single_cluster','kmeans_3_cluster','kmeans_5_cluster','dbscan_1_4_cluster',
                'dbscan_1_5_cluster','kmeans_pca_cluster']

#ca_all[output_fields].to_csv('/work/output/ca_unsupervised_min_max.csv',index=False)

### What's the deal with Alpine County?

It's mostly national forest...
https://en.wikipedia.org/wiki/Alpine_County,_California

The cells below show it's population compared to other counties

In [ ]:
data = pd.read_csv('/work/cleaned-csvs/ca_counties_full_dataset.csv')

In [ ]:
alt.Chart(data).mark_bar().encode(
    y='county_name:N',
    x='total_population:Q',
    #color='kmeans_pca_cluster:N'
)#.properties(title='K-Means With 3 Clusters on PCA Results - 2018 Data')

In [ ]:
data = data[data['year']==2018]
data[['county_name','total_population']].sort_values(by='total_population', ascending=True)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6c76417-5fde-42f3-8920-755838dec3fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>